# Signal extraction: AIRS-CH0

Let's see if we can decrease the size of the AIRS-CH0 data by selecting just the pixels that have signal. My idea is to sum across the image and take the ~3 or so rows that give the highest sum.

## Notebook set-up

In [1]:
# Set notebook root to project root
from helper_functions import set_project_root

set_project_root()

# Standard library imports
from pathlib import Path

# Third party imports
import h5py
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd
from matplotlib.ticker import FormatStrFormatter

# Local imports
from ariel_data_preprocessing.utils import load_masked_frames
import configuration as config

# Make sure the figures directory exists
figures_dir = f'{config.FIGURES_DIRECTORY}/signal_extraction'
Path(figures_dir).mkdir(parents=True, exist_ok=True)

config.COLORMAP

# Define sample frame for analysis
sample_frame = 3

Working directory: /mnt/arkk/kaggle/ariel-data-challenge


## 1. Load test frame

In [2]:
# Load uncorrected AIRS-CH0 signal for test planet
planet_path = f'{config.RAW_DATA_DIRECTORY}/train/{config.SAMPLE_PLANET}'

uncorrected_airs_signal = pd.read_parquet(
    f'{planet_path}/AIRS-CH0_signal_0.parquet'
).to_numpy().reshape(11250, 32, 356)[:, :, 39:321]

# Load corrected data and the hot/dead pixel mask for same planet from HDF5 file
with h5py.File(f'{config.SIGNAL_CORRECTED_DIRECTORY}/train.h5', 'r') as hdf:
    corrected_airs_signal = load_masked_frames(hdf, config.SAMPLE_PLANET, 'AIRS-CH0')

print(f'Uncorrected data shape: {uncorrected_airs_signal.shape}')
print(f'Corrected data shape: {corrected_airs_signal.shape}')

KeyError: "Unable to synchronously open object (object '342072318' doesn't exist)"

In [ ]:
# Plot the first frame of each
uncorrected_frame = uncorrected_airs_signal[(sample_frame * 2) - 1] # Take the 5th frame (long exposure from the 3rd CDS pair)
corrected_frame = corrected_airs_signal[sample_frame] # Take the 3rd corrected CDS exposure

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(config.STD_FIG_WIDTH, 3))

axs[0].set_title(f'Uncorrected AIRS-CH0 signal for planet {config.SAMPLE_PLANET}, frame {sample_frame}')
axs[0].imshow(uncorrected_frame, aspect='auto', cmap=config.AIRS_HEATMAP_CMAP)
axs[0].axis('off')

axs[1].set_title(f'Corrected AIRS-CH0 signal for planet {config.SAMPLE_PLANET}, frame {sample_frame}')
axs[1].imshow(corrected_frame, aspect='auto', cmap=config.AIRS_HEATMAP_CMAP)
axs[1].axis('off')

fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.1-corrected_vs_uncorrected_frames.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

Looks like we are loosing the pixel mask when we save the data to HDF5. See NumPy docks on [masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html).

## 2. Total flux by frame y-index

In [ ]:
# Sum the frames' rows
uncorrected_frame_sum = np.sum(uncorrected_frame, axis=1)
corrected_frame_sum = np.sum(corrected_frame, axis=1)

# Standardize the sums
uncorrected_frame_sum = (uncorrected_frame_sum - np.mean(uncorrected_frame_sum)) / np.std(uncorrected_frame_sum)
corrected_frame_sum = (corrected_frame_sum - np.mean(corrected_frame_sum)) / np.std(corrected_frame_sum)

In [ ]:
plt.figure(figsize=(config.STD_FIG_WIDTH, config.STD_FIG_WIDTH * 0.6))
plt.title(f'AIRS-CH0 pixel row sums, corrected vs uncorrected ' +
    f'\nPlanet {config.SAMPLE_PLANET}, frame 1')

plt.scatter(
    range(len(uncorrected_frame_sum)),
    uncorrected_frame_sum,
    color=config.LIGHT_GRAY,
    label='Uncorrected'
)

plt.scatter(
    range(len(corrected_frame_sum)),
    corrected_frame_sum,
    color=config.TRANSIT_COLOR,
    label='Corrected'
)

plt.xlabel('Row index')
plt.ylabel('Total flux (standardized)')
plt.legend()
plt.grid()
plt.savefig(
    f'{figures_dir}/02.3.2-corrected-uncorrected-total_flux_by_row.jpg',
    dpi=config.STD_FIG_DPI, bbox_inches='tight'
)
plt.show()

Wow, well, that's an interesting result right there - the corrected frame looks cleaner as an image, but after standardization the total signal across rows is almost identical to the uncorrected frame.

In [ ]:
fig = plt.figure(figsize=(config.STD_FIG_WIDTH * 1.5, config.STD_FIG_WIDTH/2))

fig.suptitle(f'AIRS-CH0 signal extraction, planet {config.SAMPLE_PLANET}, frame {sample_frame}')

gs = fig.add_gridspec(1,4)
ax1 = fig.add_subplot(gs[0, :3])
ax2 = fig.add_subplot(gs[0, 3], sharey=ax1)
plt.setp(ax2.get_yticklabels(), visible=False)

ax1.set_title(f'AIRS-CH0 signal')
ax1.imshow(corrected_frame, aspect='auto', cmap=config.AIRS_HEATMAP_CMAP)
ax1.set_xlabel('Wavelength index')
ax1.set_ylabel('Frame y-index\n(detector row)')

ax2.set_title('Row sum')
ax2.plot(
    corrected_frame_sum,
    range(len(corrected_frame_sum)),
    color=config.TRANSIT_COLOR
)

ax2.set_xlabel('Total flux (standardized)')
ax2.grid()

fig.tight_layout()
fig.savefig(
    f'{figures_dir}/02.3.3-total_flux_by_row_spectrogram.jpg',
    dpi=config.STD_FIG_DPI, bbox_inches='tight'
)



The part of the frame that actually contains the signal is clearly between pixels 10 and 20. The highest signal rows look like 14, 15, 16 and 17. We can definitely pull them out, no problem. Let's take a look.

## 3. Signal 'strip': brightest rows

I think what we should do is average or sum the top few rows and just take that forward as the 'signal'. Doing so will give us a huge data reduction. Question is - does the signal move around during the time course of exposures? It seems like in real telescope data, it would. But maybe in this synthetic dataset it doesn't. If it doesn't, we just need to look at one frame per planet, pick the rows, and then slice them out of the whole sequence. Easy. If it does move, we need to check the row sums in each frame. We can probably do that with some Numpy indexing and slicing, but it might involve a loop, so I'd rather not if I don't have to.

Let's try summing the same top six brightest rows in all frames and then make the exoplanet transit plot for the signal strip and the total data. If the plot looks wonky, we know the signal probably moved.

In [ ]:
n_rows = 6

# Sum across axis 2
row_sums = np.sum(corrected_airs_signal, axis=2)

# Get top n rows for each frame
top_rows = np.argsort(row_sums, axis=1)[:, -n_rows:]

# Now index the original array to get the top rows for each frame
signal_strip = corrected_airs_signal[:, top_rows[1], :]

# Sum the top rows per frame
signal_strip = np.sum(signal_strip, axis=1)

# Calculate total flux for each frame
total_flux = [np.sum(frame) for frame in corrected_airs_signal]

# Calculate total flux in the averaged signal strip for each frame
total_strip_flux = [np.sum(frame) for frame in signal_strip]

In [ ]:
fig = plt.figure(figsize=(config.STD_FIG_WIDTH * 2, config.STD_FIG_WIDTH))

fig.suptitle(f'AIRS-CH0 signal extraction, planet ' +
    f'{config.SAMPLE_PLANET}\nTop {n_rows} rows selected frame {sample_frame}')

gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])

ax1.set_title(f'Total signal')

ax1.scatter(
    list(range(len(total_flux))),
    total_flux,
    color='black',
    s=0.5
)

ax1.set_xlabel('Frame number')

y_lims = ax1.get_ylim()
y_range = y_lims[1] - y_lims[0]
data_mean = np.mean(total_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

ax1.set_yticks(np.round(np.linspace(y_start, y_end, 5), 2))
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2e'))
ax1.set_ylabel(f'Total flux (range: {y_range:.2e})')
ax1.grid()

ax2.set_title(f'Extracted signal')

ax2.scatter(
    list(range(len(total_strip_flux))),
    total_strip_flux,
    color=config.TRANSIT_COLOR,
    s=0.5
)

ax2.set_xlabel('Frame number')
ax2.grid()

data_mean = np.mean(total_strip_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

ax2.set_yticks(np.linspace(y_start, y_end, 5))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2e'))

ax3.set_title(f'Extracted region')
ax3.imshow(corrected_frame, aspect='auto', cmap=config.AIRS_HEATMAP_CMAP)
ax3.axhline(y=np.max(top_rows[0]) + 1, color=config.GREEN, linestyle='-')
ax3.axhline(y=np.min(top_rows[0]) - 1, color=config.GREEN, linestyle='-', label='Extraction bounds')
ax3.set_xlabel('Wavelength index')
ax3.set_ylabel('Frame y-index\n(detector row)')
ax3.legend(framealpha=1)
fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.4-transit_plot_total_vs_strip.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight')

Looks great. The signal strip definitely didn't wander off during the exposure time series. The transit signature is just slightly deeper in the whole frame trace than the signal strip trace. This makes sense; there is some signal left in the rows we excluded. There is a slight qualitative increase in noise when summing the strip relative to the input data. Makes me think that maybe the signal strip is moving around a tiny, sub-pixel bit. It makes sense that in real data, when satellite drifts or the star moves, the alignment camera brings it back, resulting in the signal floating around just a bit? Don't know.

Let's select the top six rows individually in each image and see what that does. If there is any movement in the signal strip, this will fix it. And let's do it without a loop, if possible.

In [ ]:
# Sum across axis 2
row_sums = np.sum(corrected_airs_signal, axis=2)

# Get top n rows for each frame
top_rows = np.argsort(row_sums, axis=1)[:, -n_rows:]

# Now index the original array to get the top rows for each frame
signal_strip = corrected_airs_signal[:, top_rows[1], :]

# And finally average across the rows to get a 1D signal per frame  
signal_strip = np.sum(signal_strip, axis=1)

# Sum the strip flux for each frame
total_strip_flux = [np.sum(frame) for frame in signal_strip]

Geesh, that wasn't so bad after all - thanks Numpy! No loops, just indexing and slicing. How `bout now?

In [ ]:
fig, axs = plt.subplots(
    1,
    2,
    figsize=(
        config.STD_FIG_WIDTH * 2,
        config.STD_FIG_WIDTH - 2
    )
)

fig.suptitle(f'AIRS-CH0 signal extraction, planet {config.SAMPLE_PLANET}' +
    f'\nTop {n_rows} rows selected each frame')


axs[0].set_title(f'Total AIRS-CH0 signal')

axs[0].scatter(
    list(range(len(total_flux))),
    total_flux,
    color='black',
    s=0.5
)

axs[0].set_xlabel('Frame number')
axs[0].grid()

y_lims = axs[0].get_ylim()
y_range = y_lims[1] - y_lims[0]
data_mean = np.mean(total_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

axs[0].set_yticks(np.round(np.linspace(y_start, y_end, 5), 2))
axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2e'))
axs[0].set_ylabel(f'Total flux (range: {y_range:.2e})')


axs[1].set_title(f'Extracted AIRS-CH0 signal')

axs[1].scatter(
    list(range(len(total_strip_flux))),
    total_strip_flux,
    color='black',
    s=0.5
)

axs[1].set_xlabel('Frame number')
axs[1].grid()

data_mean = np.mean(total_strip_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

axs[1].set_yticks(np.linspace(y_start, y_end, 5))
axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.2e'))


fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.5-transit_plot_total_vs_strip_each_frame.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

That did nothing - looking at the points closely, I can tell this plot is the same as the first one.

I have another idea - let's try subtracting a background strip, where the signal isn't, from each frame. How about we take two background strips and average them, say at pixels 3, 4 and 5 and 25, 26 and 27, so the same number of pixels as are aggregated in the signal strip but above and below the signal strip in the frame.

## 4. Frame background correction

In [ ]:
# Get average background strip
background_strip = corrected_airs_signal[:, [3, 4, 5, 25, 26, 27], :]
background_strip = np.sum(background_strip, axis=1)
total_background_flux = np.sum(background_strip, axis=1)

# Subtract from the signal strip
corrected_signal_strip = signal_strip - background_strip
total_strip_flux = np.sum(corrected_signal_strip, axis=1)

In [ ]:
fig = plt.figure(figsize=(config.STD_FIG_WIDTH * 2, config.STD_FIG_WIDTH))

fig.suptitle(f'AIRS-CH0 background corrected signal extraction, planet {config.SAMPLE_PLANET}')

gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])

ax1.set_title(f'Total AIRS-CH0 signal')

ax1.scatter(
    list(range(len(total_flux))),
    total_flux,
    color='black',
    s=0.5
)

ax1.set_xlabel('Frame number')

y_lims = ax1.get_ylim()
y_range = y_lims[1] - y_lims[0]
data_mean = np.mean(total_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

ax1.set_yticks(np.round(np.linspace(y_start, y_end, 5), 2))
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2e'))
ax1.set_ylabel(f'Total flux (range: {y_range:.2e})')
ax1.grid()

ax2.set_title(f'Extracted AIRS-CH0 signal, background corrected')

ax2.scatter(list(
    range(len(total_strip_flux))),
    total_strip_flux,
    color='black',
    s=0.5
)

ax2.set_xlabel('Frame number')

data_mean = np.mean(total_strip_flux)
y_start = data_mean - y_range/2
y_end = data_mean + y_range/2

ax2.set_yticks(np.linspace(y_start, y_end, 5))
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2e'))
ax2.set_ylabel(f'Total flux (range: {y_range:.2e})')
ax2.grid()

ax3.set_title(f'Extracted AIRS-CH0 background')

ax3.scatter(
    list(range(len(total_background_flux))),
    total_background_flux,
    color=config.GRAY,
    s=0.5
)
ax3.set_xlabel('Frame number')
ax3.set_ylim(1.3e5,1.5e5)
ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2e'))
ax3.grid()

fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.6-transit_plot_total_vs_strip_background_corrected.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight')

The strip signal still looks a tiny bit more noisy, and we have introduced a few low outliers. These should be easy to threshold out, if we decide we want to use this approach. But, I don't think we should - there is clearly signal left in the 'background' frames so subtracting it will cause us to loose signal.

**Results so far**

1. Summing the top few brightest rows preserves the transit signal beautifully.
2. The top rows for a planet can be selected by looking at the first frame only.
3. Subtracting a per-frame background from the signal strip is not necessary.

Let's remember the goal here - we are going for data reduction without loss of signal, and I think we accomplished that already! Here's the size of the dataset before and after extraction of the top 4 brightest rows:

In [ ]:
starting_size = corrected_frame.shape[0] * corrected_frame.shape[1] * corrected_airs_signal.shape[0]
extracted_size = signal_strip.shape[0] * signal_strip.shape[1]

print(f'Size of the dataset before extraction: {starting_size} pixels')
print(f'Size of the dataset after extraction: {extracted_size} pixels')

print(f'Data reduction factor: {starting_size / extracted_size:.1f}x')

Killer result - we should probably be doing this before the preprocessing. I bet we could get the pipeline runtime down to minutes instead of hours! But, I want to try a few more things before we go implement that.

Let's think about how to select the rows to include in the strip - how many do we need to include? This decision should be made per-star to account for variations in brightness. First, let's look at a bunch of rows individually and see how the signal changes as we move away from the brightest row. This is without the background subtraction.

## 5. Top rows by brightness: spectra

In [ ]:
# Number of rows to extract
n_rows = 11

# Get the sorted indices of the rows based on the total flux
corrected_indices = np.argsort(corrected_frame_sum)

# Take the top n rows in descending order
top_rows = corrected_indices[-n_rows:][::-1]

In [ ]:
fig, axs = plt.subplots(
    n_rows,
    1,
    sharex=True,
    sharey=True,
    figsize=(config.STD_FIG_WIDTH * 2, n_rows * (config.STD_FIG_WIDTH / 4))
)

axs = axs.flatten()

fig.suptitle(f'AIRS-CHO top {n_rows} brightest rows {config.SAMPLE_PLANET}, frame {sample_frame}')

for i in range(n_rows):

    uncorrected_signal = uncorrected_frame[top_rows[i], :]
    uncorrected_signal = (
        uncorrected_signal - np.mean(uncorrected_signal)
    ) / np.std(uncorrected_signal)

    corrected_signal = corrected_frame[top_rows[i], :]
    corrected_signal = (
        corrected_signal - np.mean(corrected_signal)
    ) / np.std(corrected_signal)

    axs[i].scatter(
        range(len(uncorrected_signal)),
        uncorrected_signal,
        color='black',
        s=16,
        label='Uncorrected'
    )

    axs[i].plot(
        range(len(corrected_signal)),
        corrected_signal,
        color=config.ORANGE,
        label='Corrected'
    )
    
    axs[i].set_ylabel(f'Row {top_rows[i]}\nsignal')
    axs[i].grid()

axs[0].legend(loc='upper left', markerscale=3)
axs[-1].set_xlabel('Wavelength index')

fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.7-top_{n_rows}_rows_signal.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

OK, I take it back - the corrected signal appears less noisy. Interestingly, the overall subjective noise seems to be a function of the total signal. Lower total signal across the row means more noise. Rows 20, 21 and 11 are the dimmest and their raw data also has the most jitter, followed in order by row 12.

Looks like the noise in the uncorrected signal really starts to come in after the 4th brightest row. Looking at the total brightness plot and the images, this makes sense - this is where the signal has really dropped off.

Let's make the transit plot for the same rows:

## 6. Top rows by brightness: transits

In [ ]:
fig, axs = plt.subplots(4, 3, sharey=True, figsize=(config.STD_FIG_WIDTH * 2, config.STD_FIG_WIDTH * 2))
axs = axs.flatten()

fig.suptitle(f'AIRS-CH0 total signal over time, planet {config.SAMPLE_PLANET}')

standardized_total_flux = (total_flux - np.mean(total_flux)) / np.std(total_flux)

axs[0].set_title(f'Whole frame')

axs[0].scatter(
    list(range(len(standardized_total_flux))),
    standardized_total_flux,
    color='black',
    s=0.5
)

axs[0].set_ylim(-3, 2)
axs[0].set_xlabel('Frame number')
axs[0].set_ylabel('Total signal (standardized)')
axs[0].grid()

for i, row in enumerate(top_rows):

    signal_strip = corrected_airs_signal[:, row, :]

    total_strip_flux = [np.sum(frame) for frame in signal_strip]
    total_strip_flux = (
        total_strip_flux - np.mean(total_strip_flux)
    ) / np.std(total_strip_flux)

    axs[i+1].set_title(f'Number {i + 1} brightest row')

    axs[i+1].scatter(
        list(range(len(total_strip_flux))),
        total_strip_flux,
        color='black',
        s=0.5
    )

    axs[i+1].set_xlabel('Frame number')
    axs[i+1].grid()

    if (i + 1) % 3 == 0:
        axs[i+1].set_ylabel('Total signal (standardized)')

fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.8-top_{n_rows}_rows_transit.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

Ok - so single rows are messy relative to summed rows or the total signal, and the noise seems to increase then decrease slightly as we work our way down the total flux ranking. It kinda bothers me that this is the case, I don't have a good explanation. When I combined rows, I used the sum, not the average - it seems to suggest that the noise in the rows is correlated (or anti-correlated). Such that when you add them up they sum to the same-ish amount of total flux across the frames. That feels like a synthetic data artifact - but I guess it might be some property of the detector?

Last thing to try - lets see how things look if we sum different numbers of rows. Also try standardizing each row before summing.

## 7. Top row sums: spectra

In [ ]:
row_nums = [1,2,3,4,5,6,7,8,9,10]

fig, axs = plt.subplots(
    len(row_nums),
    2,
    sharex=True,
    sharey=True,
    figsize=(
        config.STD_FIG_WIDTH * 2,
        len(row_nums) * (config.STD_FIG_WIDTH / 6)
    )
)

fig.suptitle(f'AIRS-CH0 row sum spectra for planet {config.SAMPLE_PLANET}, frame {sample_frame}')
fig.supylabel('Total signal (standardized)')
fig.supxlabel('Wavelength index')

for i, n_rows in enumerate(row_nums):

    row_signals = corrected_frame[top_rows[:n_rows], :]
    standardized_row_signals = (
        row_signals - np.mean(row_signals, axis=1, keepdims=True)
    ) / np.std(row_signals, axis=1, keepdims=True)

    row_signals = np.sum(row_signals, axis=0)
    standardized_row_signals = np.sum(standardized_row_signals, axis=0)

    row_signals = (row_signals - np.mean(row_signals)) / np.std(row_signals)
    standardized_row_signals = (
        standardized_row_signals - np.mean(standardized_row_signals)
    ) / np.std(standardized_row_signals)

    axs[i,0].scatter(
        range(len(row_signals)),
        row_signals,
        color=config.SPECTRUM_COLOR,
        s=4,
        label='Uncorrected'
    )

    axs[i,1].scatter(
        range(len(standardized_row_signals)),
        standardized_row_signals,
        color=config.BLUE,
        s=4,
        label='Standardized before summing'
    )

    axs[i,0].set_ylabel(f'Top {n_rows} rows')
    axs[i,0].grid()
    axs[i,1].grid()

axs[0,0].legend(loc='upper left', markerscale=5)
axs[0,1].legend(loc='upper left', markerscale=5)

fig.tight_layout()
fig.savefig(
    f'{figures_dir}/02.3.9-top_{len(row_nums)}_rows_summed_spectrum.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

## 8. Top row sums: transits

In [ ]:
row_nums = [1,2,3,4,5,6,7,8,9,10]

fig, axs = plt.subplots(
    len(row_nums),
    2,
    sharex=True,
    sharey=True,
    figsize=(config.STD_FIG_WIDTH * 2,
    len(row_nums) * (config.STD_FIG_WIDTH / 4.5))
)

fig.suptitle(f'AIRS-CH0 total signal over time, planet {config.SAMPLE_PLANET}')
fig.supylabel('Total signal (standardized)')
fig.supxlabel('Frame')

for i, n_rows in enumerate(row_nums):

    row_signals = corrected_airs_signal[:, top_rows[:n_rows], :]

    # Standardize each wavelength across the frames
    standardized_row_signals = (
        row_signals - np.mean(row_signals, axis=2, keepdims=True)
    ) / np.std(row_signals, axis=2, keepdims=True)

    total_rows_signal = [np.sum(frame) for frame in row_signals]
    total_standardized_rows_signal = [np.sum(frame) for frame in standardized_row_signals]

    total_rows_signal = (
        total_rows_signal - np.mean(total_rows_signal)
    ) / np.std(total_rows_signal)

    total_standardized_rows_signal = (
        total_standardized_rows_signal - np.mean(total_standardized_rows_signal)
    ) / np.std(total_standardized_rows_signal)

    axs[i,0].scatter(
        range(len(total_rows_signal)),
        total_rows_signal,
        color=config.TRANSIT_COLOR,
        s=4,
        label='Uncorrected'
    )
    axs[i,1].scatter(
        range(len(total_standardized_rows_signal)),
        total_standardized_rows_signal,
        color=config.GREEN,
        s=4,
        label='Standardized before summing'
    )
    axs[i,0].set_ylabel(f'Top {n_rows} rows')
    axs[i,0].grid()
    axs[i,1].grid()

axs[0,0].legend(loc='upper left', markerscale=5)
axs[0,1].legend(loc='upper left', markerscale=5)

fig.tight_layout()
fig.savefig(
    f'{figures_dir}/02.3.10-top_{len(row_nums)}_rows_summed_transit.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

**Results so far:**

1. Summing the top few brightest rows preserves the transit signal beautifully.
2. The top rows for a planet can be selected by looking at the first frame only.
3. Subtracting a per-frame background from the signal strip is not necessary.
4. ~4 rows seems to be the sweet spot (at least for this planet) after than the underlying data starts to get really noisy.
5. Don't standardize the rows before summing them.


## 9. Combined row sum plot: transits & spectra

In [ ]:
row_nums = [1,2,3,4,5]

fig, axs = plt.subplots(
    len(row_nums) + 1,
    2,
    sharex='col',
    sharey='col',
    figsize=(
        config.STD_FIG_WIDTH * 2,
        len(row_nums) * (config.STD_FIG_WIDTH / 4.5)
    )
)

fig.suptitle(f'AIRS-CH0 top n row sums comparison, planet {config.SAMPLE_PLANET}')
fig.supylabel('Total signal (standardized)')

standardized_total_flux = (total_flux - np.mean(total_flux)) / np.std(total_flux)

axs[0,0].set_title('Total signal over time')

axs[0,0].scatter(
    range(len(standardized_total_flux)),
    standardized_total_flux,
    color='black',
    s=4
)

axs[0,0].grid()

axs[0,1].set_title(f'Total signal by wavelength index, frame {sample_frame}')

total_signal = np.sum(corrected_frame, axis=0)
standardized_total_signal = (total_signal - np.mean(total_signal)) / np.std(total_signal)

axs[0,1].scatter(
    range(len(standardized_total_signal)),
    standardized_total_signal,
    color='black',
    s=4
)

axs[0,1].grid()

axs[0,0].set_ylabel(f'Total signal')

for i, n_rows in enumerate(row_nums):

    row_signals = corrected_airs_signal[:, top_rows[:n_rows], :]

    # Standardize each wavelength across the frames
    standardized_row_signals = (
        row_signals - np.mean(row_signals, axis=2, keepdims=True)
    ) / np.std(row_signals, axis=2, keepdims=True)

    total_rows_signal = [np.sum(frame) for frame in row_signals]
    total_standardized_rows_signal = [np.sum(frame) for frame in standardized_row_signals]
    total_rows_signal = (total_rows_signal - np.mean(total_rows_signal)) / np.std(total_rows_signal)

    axs[i+1, 0].scatter(
        range(len(total_rows_signal)),
        total_rows_signal,
        color=config.TRANSIT_COLOR,
        s=4,
        label='Uncorrected'
    )

    axs[i+1, 0].grid()

    row_signals = corrected_frame[top_rows[:n_rows], :]
    row_signals = np.sum(row_signals, axis=0)
    row_signals = (row_signals - np.mean(row_signals)) / np.std(row_signals)

    axs[i+1, 1].scatter(
        range(len(row_signals)),
        row_signals,
        color=config.SPECTRUM_COLOR,
        s=4,
        label='Uncorrected'
    )

    axs[i+1, 1].grid()

    axs[i+1, 0].set_ylabel(f'Top {n_rows} rows')

axs[-1, 0].set_xlabel('Frame')
axs[-1, 1].set_xlabel('Wavelength index')

fig.tight_layout()

fig.savefig(
    f'{figures_dir}/02.3.11-top_{len(row_nums)}_rows_summed_transits-spectra.jpg',
    dpi=config.STD_FIG_DPI,
    bbox_inches='tight'
)

Before we move on, let's come up with a way to automatically select the number of rows based on a fraction or percent of signal. Imaging drawing a horizontal line on the total flux by frame y-index plot from the start of this notebook. That way we don't need to worry about loosing signal by selecting too few rows for very bright stars or vice versa.

## 10. Automated row selection

Idea is to shift the row sums so that the minimum is zero, then we can select rows by a fraction of the maximum value.

In [ ]:
# Sum the frame's rows
row_sums = np.sum(corrected_frame, axis=1)

# Shift the sums so the minimum is zero
row_sums -= np.min(row_sums)
signal_range = np.max(row_sums)
inclusion_thresholds = [0.75, 0.50, 0.25, 0.10]

plt.figure(figsize=(config.STD_FIG_WIDTH, config.STD_FIG_WIDTH * 0.6))
plt.title(f'AIRS-CH0 row selection, planet {config.SAMPLE_PLANET}, frame {sample_frame}')

plt.scatter(
    range(len(row_sums)),
    row_sums,
    color=config.TRANSIT_COLOR,
    label='Signal'
)

for i, threshold in enumerate(inclusion_thresholds):
    plt.hlines(
        y=signal_range * threshold,
        xmin=0,
        xmax=len(row_sums),
        colors=config.COLORS[8+i],
        linestyles='dashed',
        label=f'{int(threshold*100)}% of max'
    )

plt.xlabel('Row index')
plt.ylabel('Total flux (shifted)')
plt.grid()
plt.legend()

plt.savefig(
    f'{figures_dir}/02.3.12-row_number_selection.jpg',
    dpi=config.STD_FIG_DPI, bbox_inches='tight'
)
plt.show()

You get the idea - this will work for any planet - we might not get a constant number of rows, but we should get a consistent - ish fraction of the signal overall signal. Our minimum sweet spot for the sample planet was 4 rows, using this cutoff method, that translates to 25% max signal inclusion threshold. Honestly, that bothers me a bit too - the signal we are looking for is such a small fraction to begin with, it seems like a bad idea to throw signal away. But maybe cleaner is better.